# Build RNN to generate hunger games sounding text

based on tutorial in here : https://www.tensorflow.org/text/tutorials/text_generation

Could not use all features since I have tf version 2.5

In [1]:
import sys, os
import tensorflow as tf
import pandas as pd
import numpy as np

Init Plugin
Init Graph Optimizer
Init Kernel


In [109]:
tf.__version__

'2.5.0'

In [5]:
with open("(1) The Hunger Games.txt", 'rb') as file : 
    txt_file = file.readlines()
type(txt_file)

list

In [11]:
len(txt_file)

1071

In [25]:
txt_all = [str(x)[2:] for x in txt_file][1:]
txt_all_long = [x.replace('\\r','\\s').replace('\\n','\\s') 
                for x in txt_all if len(x) > 49]

487

In [34]:
single_text = ' '.join(txt_all_long)
vocab = sorted(set(single_text))
len(vocab)

74

In [36]:
in_size, out_size = 200, 100
def split_input_sequence(seq) : 
    """
    split sequence into all but last & all but first
    """
    input_text = seq[:-1]
    output_text = seq[1:]
    return input_text, output_text


In [43]:
uc_split = tf.strings.unicode_split(single_text, input_encoding='UTF-8')
data = tf.data.Dataset.from_tensor_slices(uc_split)

In [88]:
vocab_lookup = {str(x):i for i,x in enumerate(list(vocab))}
letter_lookup = {i:str(x) for i,x in enumerate(list(vocab))}
uc_np = pd.Series(uc_split.numpy().astype(str)).astype(str).replace(vocab_lookup).values
tens = tf.convert_to_tensor(uc_np, dtype=tf.int64)

In [89]:
ids_dataset = tf.data.Dataset.from_tensor_slices(tens)

In [90]:
def translate(n) : 
    """
    translate
    """
    return letter_lookup[n]
for ids in ids_dataset.take(10):
    print(translate(ids.numpy()))

W
h
e
n
 
I
 
w
a
k


In [98]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1) : 
    print(seq)

tf.Tensor(
[44 55 52 61  0 30  0 70 48 58 52  0 68 63  5  0 67 55 52  0 62 67 55 52
 65  0 66 56 51 52  0 62 53  0 67 55 52  0 49 52 51  0 56 66  0 50 62 59
 51  7  0 34 72  0 53 56 61 54 52 65 66  0 66 67 65 52 67 50 55  0 62 68
 67  5  0 66 52 52 58 56 61 54  0 37 65 56 60 66  0 70 48 65 60 67 55  0
 49 68 67  0 53], shape=(101,), dtype=int64)


In [102]:
dataset = sequences.map(split_input_sequence)

for input_example, target_example in dataset.take(1):
    print("Input :", (input_example).numpy())
    print("Target:", (target_example).numpy())

Input : [44 55 52 61  0 30  0 70 48 58 52  0 68 63  5  0 67 55 52  0 62 67 55 52
 65  0 66 56 51 52  0 62 53  0 67 55 52  0 49 52 51  0 56 66  0 50 62 59
 51  7  0 34 72  0 53 56 61 54 52 65 66  0 66 67 65 52 67 50 55  0 62 68
 67  5  0 66 52 52 58 56 61 54  0 37 65 56 60 66  0 70 48 65 60 67 55  0
 49 68 67  0]
Target: [55 52 61  0 30  0 70 48 58 52  0 68 63  5  0 67 55 52  0 62 67 55 52 65
  0 66 56 51 52  0 62 53  0 67 55 52  0 49 52 51  0 56 66  0 50 62 59 51
  7  0 34 72  0 53 56 61 54 52 65 66  0 66 67 65 52 67 50 55  0 62 68 67
  5  0 66 52 52 58 56 61 54  0 37 65 56 60 66  0 70 48 65 60 67 55  0 49
 68 67  0 53]


2022-07-05 13:29:39.019665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-05 13:29:39.021647: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [103]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [183]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 512

class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [184]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [185]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 74) # (batch_size, sequence_length, vocab_size)


In [186]:
model.summary()

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  18944     
_________________________________________________________________
gru_3 (GRU)                  multiple                  1182720   
_________________________________________________________________
dense_3 (Dense)              multiple                  37962     
Total params: 1,239,626
Trainable params: 1,239,626
Non-trainable params: 0
_________________________________________________________________


### Untrained model prediction

In [187]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([27, 29,  1, 54, 15, 64,  6, 37, 38,  0, 45, 27, 16, 28,  3, 55, 47,
       15,  9, 27, 59, 40, 73, 61, 70, 30, 21,  9,  8, 51, 10, 51, 11, 19,
       45, 58, 10, 12, 53,  6, 22, 36, 38, 62, 59, 10, 28,  3, 42, 35, 23,
       36, 23, 56, 11, 62, 40,  9, 18, 43, 30, 70, 41, 41, 62, 36, 45, 14,
        1, 61,  9, 32, 12, 26,  3, 31, 48, 19, 68,  0, 35, 71, 36, 16,  0,
       12, 72,  7, 27, 54, 65, 54, 45, 12, 57, 43, 53, 35, 14, 20])

In [188]:
def translate_array(arr) : 
    """
    translate over & over on an array
    """
    return np.array([translate(a) for a in arr])
def translate_array_join(arr) : 
    """
    join the text
    """
    return ''.join(translate_array(arr))

In [189]:
print("Input:\n", translate_array_join(input_example_batch[0].numpy()))
print()
print("Next Char Predictions:\n", translate_array_join(sampled_indices))

Input:
  unintelligible as they babble out praise. As I glance around, I notice a lot of the other tributes 

Next Char Predictions:
 FH!g6q-PQ XF7G'h\60FlSznwI?0/d1d2:Xk13f-AOQol1G'UNBOBi2oS09VIwTToOX5!n0K3E'Ja:u NxO7 3y.FgrgX3jVfN5;


### Train model

In [190]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)
EPOCHS = 25
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25


2022-07-05 15:05:21.982729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-07-05 15:05:22.331129: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-07-05 15:05:22.607593: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - 14s 151ms/step - loss: 2.9650
Epoch 2/25
80/80 [==============================] - 12s 151ms/step - loss: 2.4961
Epoch 3/25
80/80 [==============================] - 12s 151ms/step - loss: 2.5215
Epoch 4/25
80/80 [==============================] - 12s 149ms/step - loss: 2.4581
Epoch 5/25
80/80 [==============================] - 12s 149ms/step - loss: 2.2793
Epoch 6/25
80/80 [==============================] - 12s 149ms/step - loss: 2.1813
Epoch 7/25
80/80 [==============================] - 12s 149ms/step - loss: 2.0795
Epoch 8/25
80/80 [==============================] - 12s 149ms/step - loss: 1.9714
Epoch 9/25
80/80 [==============================] - 12s 149ms/step - loss: 1.8405
Epoch 10/25
80/80 [==============================] - 12s 150ms/step - loss: 1.7640
Epoch 11/25
80/80 [==============================] - 12s 151ms/step - loss: 1.6915
Epoch 12/25
80/80 [==============================] - 13s 156ms/step - loss: 1.6198
Epoch 13/25
80/80 [=====

In [191]:
x,y = model(inputs=tf.convert_to_tensor([[0,2,1]*30,[1,2]*45],dtype=tf.int64),states=None,return_state=True)
x.shape, y.shape

(TensorShape([2, 90, 74]), TensorShape([2, 512]))

In [192]:
translate_array_join(tf.squeeze(tf.random.categorical(x[:, -1, :], num_samples=1), axis=-1).numpy())

"\\'"

In [193]:
def split_to_tens(split) : 
    """
    take unicode split array & turn to tensor
    """
    uc_np = pd.DataFrame(split.numpy().astype(str)).astype(str).replace(vocab_lookup).values
    tens = tf.convert_to_tensor(uc_np, dtype=tf.int64)
    return tens

In [196]:
states = None
next_char = tf.constant(['Good morning puppy dog, '])
result = [next_char]
for i in range(1000) : 
    nextchar = tf.strings.unicode_split(next_char, 'UTF-8')
    ids = split_to_tens(nextchar)
    logits, states = model(inputs=ids, states=states,return_state=True)
    predicted_logits = logits[:, -1, :]
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1).numpy()
    next_char = translate_array_join(predicted_ids)
    result.append(next_char)
results = tf.strings.join(result)
results

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'Good morning puppy dog, tided, not atvayled this io my bow alones back, virective from 1ever Cinna st cking and thentsext in his bagglervgueensy I am time to . .s. The sure beitures you can see the Careers forted about of riceat on them until thatfaci wisholights to my boot to the neishat. Because I gnt chinking then you have to drame is, without unatlit eating the paured in my facious berrated the squeeuses. All right? Bur why cakes are said and piss a hay. Handinces. How we war through the mayorr limued intlly of ricking out the cannon quemertifiel gnot tsay of waret of all the trumpets. Of Efthe takes about my head and I bread must position of creeks. It did, seppang her finished and distrestition. Of course, I turn pinkled the morning sincans. Youre my streawn, Im siredy spreads this out of survivated. So she makes it is you, Katniss straight. Im going my head and pussed losides the Careers only as it was knomb? admer? all old s

In [168]:
predicted_ids

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

# LSTM